## Step 1: Setup Environment Variables

In [1]:
# Make sure to have .env with GOOGLE_API_KEY
from dotenv import load_dotenv

load_dotenv("../../.env")
print("✅ Environment loaded")

✅ Environment loaded


## Step 2: Define Tools

In [2]:
from langchain_core.tools import tool

In [3]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}"
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

In [4]:
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

✅ Tools defined


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

## Step 3: Initialize LLM and Bind Tools

In [6]:
# Initialize LLM with tools
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

In [7]:
tools = [currency_converter, emi_calculator]

In [8]:
llm_with_tools = llm.bind_tools(tools)
print("✅ LLM initialized with tools")

✅ LLM initialized with tools


In [9]:
from langgraph.graph import MessagesState
from langgraph.graph import END
from typing import Literal

## Step 4: Define LLM Node and Router Node

In [10]:
# Define LLM node
def call_llm(state: MessagesState):
    """LLM node that invokes the LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

print("✅ LLM node defined")

✅ LLM node defined


In [11]:
# Define router function
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router that decides next step based on tool_calls."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

print("✅ Router function defined")

✅ Router function defined


## Step 5: Build or Compile Graph

In [13]:
# LangGraph and LangChain imports
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START
from langgraph.prebuilt import ToolNode

In [14]:
# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges(
    "llm",
    should_continue,
    {"tools": "tools", END: END}
)
workflow.add_edge("tools", "llm")

In [15]:
app = workflow.compile()
print("✅ Graph compiled")

✅ Graph compiled


## Step 6: Test or Validate Graph

In [16]:
# Test the graph
test_state = {
    "messages": [HumanMessage(content="Convert 100 USD to EUR")]
}

In [17]:
result = app.invoke(test_state)

In [ ]:
# User Query: Convert 100 USD to EUR
result["messages"][-1].content

'100 USD is approximately equal to 92 EUR.'